In [15]:
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import os
from langgraph.prebuilt import tool_node
import google.generativeai as genai
import fitz
from typing import TypedDict
from pprint import pprint
import json

In [16]:
class AgentType(TypedDict):
    offer_analysis: dict
    preference_alignment: dict
    upskill_recommendations: dict

In [17]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

In [18]:
def upskill_agent(state):
    offer = state["offer_analysis"]
    preference = state["preference_alignment"]

    if isinstance(offer, str):
        offer = json.loads(offer)
    if isinstance(preference, str):
        preference = json.loads(preference)

    prompt = f"""
    You are a career development AI assistant.
    Given the following job offer details and the candidate's current skills and interests,
    suggest onboarding preparation materials in this format:

    {{
      "recommended_courses": ["course 1", "course 2", ...],
      "onboarding_resources": ["resource 1", "resource 2", ...],
      "skill_gap_analysis": "Brief explanation of missing or weak skills"
    }}

    Job Offer:
    {json.dumps(offer, indent=2)}

    Candidate Profile:
    {json.dumps(preference, indent=2)}
    """

    response = model.generate_content(prompt)
    return {"upskill_recommendations": response.text}


In [19]:
builder = StateGraph(AgentType)

builder.add_node("UpskillAgent", upskill_agent)
builder.set_entry_point("UpskillAgent")
builder.set_finish_point("UpskillAgent")

graph = builder.compile()


In [20]:
result = graph.invoke({
    "offer_analysis": {'bond_terms': None,
 'job_title': 'Software Engineer',
 'joining_date': '2017-05-18',
 'location': 'Nasik',
 'salary': {'gross_annual': 314935.38,
            'monthly_breakdown': {'basic': 8000,
                                  'bonus': 6300,
                                  'conveyance': 3600,
                                  'education_allowance': 3600,
                                  'gratuity': 4615,
                                  'hra': 4800,
                                  'leave_benefit': 14000,
                                  'medical_reimbursement': 1500,
                                  'mediclaim': 1000,
                                  'pf': 960,
                                  'variable': 3000},
            'variable_component': 'Based on monthly PPI (Personal Performance '
                                  'Index) + CPI (Company Performance Index)'}},
    "preference_alignment": {'career_goals': 'Contribute to innovative projects that make a difference',
 'expected_ctc': None,
 'interests': ['Full Stack Development',
               'MERN Stack',
               'PERN Stack',
               'Scalable Web Applications',
               'Innovative Projects'],
 'preferred_location': 'Coimbatore, Tamil Nadu, India',
 'skills': ['JavaScript',
            'Nodejs',
            'fastAPI',
            'Python',
            'Express js',
            'MongoDB',
            'PostgreSQL',
            'Redis',
            'Elasticsearch',
            'RabbitMQ',
            'React js',
            'Next js',
            'Amazon S3',
            'AWS Elastic Beanstalk',
            'AWS Cloudfront',
            'Prisma ORM',
            'UI/UX',
            'Responsive Web Design']}
})

json_string = result["upskill_recommendations"].strip().replace('```json\n', '').replace('\n```', '')
formatted_dict = json.loads(json_string)
pprint(formatted_dict)

{'onboarding_resources': ['Company Internal Wiki/Documentation',
                          'Code Style Guide',
                          'Project Management Tool Tutorial (e.g., Jira, '
                          'Asana)',
                          'Git Workflow and Best Practices Guide',
                          'Introduction to the Company Culture and Values',
                          'Team Introduction and Communication Channels',
                          'Onboarding Checklist and Timeline'],
 'recommended_courses': ['Advanced React.js and Redux',
                         'Mastering Next.js',
                         'Building Scalable APIs with Node.js and Express.js',
                         'Microservices Architecture with Node.js',
                         'Optimizing Database Performance with PostgreSQL',
                         'AWS Certified Developer - Associate'],
 'skill_gap_analysis': 'While the candidate possesses a strong foundation in '
                       'full